# Wypożyczalnia sprzętu sportowego: Model relacyjnej bazy danych

**Autor**: Filip Dąbrowski </br>
**Numer indeksu**: 341057 </br>
**Wydział**: Elektryczny </br>
**Przedmiot**: Bazy Danych </br>

## Opis projektu

Poniżej przedstawiono proces implementacji bazy danych dla wypożyczalni sprzętu sportowego, zgodny z wymaganiami projektu udostępnionymi na platformie Leia.

## Modelowanie

## Model konceptualny

Model konceptualny jest modelem uproszczony. Nie wskazałem w nim kluczy obcych ani relacji wiele-do-wielu, a także zastosowałem tylko minimalną normalizację. Modelowanie konceptualne na początku projektu pomogło mi określić, jakie encje będą potrzebne, jakie mniej więcej będą miały atrybuty i jak będą się ze sobą łączyć.

![](/wypozyczalnia_model_konceptualny.png)

## Model implementacyjny

Model podzieliłem na pięć obszarów biznesowych, według których później stworzyłem schematy. Są to:
* Osoby,
* Zamówienia,
* Serwis,
* Sprzęt,
* Kategorie.

Taki podział umożliwa w systemie odpowiednie nadanie uprawnień, zwiększając bezpieczeństwo i integralność danych. Naturalnie, podział na schematy mógłby być inny i zależy od specyfiki biznesu.

### Osoby

Klientów i pracowników postanowiłem podzielić na dwie oddzielne tabele, które powiązałem z tabelą nadrzędną `Osoba`. Każdy pracownik i klient musi być osobą. Osoby mogą istnieć niezależnie i nie muszą być przypisane do żadnej z encji podrzędnych. W takim układzie możliwe jest także, że jedna osoba jest jednocześnie pracownikiem i klientem. Takie rozwiązanie było moją świadomą decyzją i zamierzonym działaniem systemu.

Alternatywnie mogłem stworzyć jedną dużą tabelę zawierającą dane zarówno klientów, jak i pracowników. Uznałem jednak, że taka tabela zawierałaby zbyt wiele wartości `NULL`, ponieważ encje podrzędne mają mało wspólnych atrybutów.

W encji `Pracownik` zastosowałem relację unarną, aby uwzględnić przełożonego w tej samej tabeli. Według mojej wiedzy, takie rozwiązanie jest powszechnie stosowane. **Świadomie zdecydowałem się nie normalizować atrybutu `Adres`**. Przyjąłem, że liczba pracowników wypożyczalni będzie stosunkowo niewielka, a czas na przygotowanie projektu miałem ograniczony, dlatego zostawiłem to pole w formie tekstu. Znormalizowałem jednak atrybuty `StatusZatrudnienia` oraz `Stanowisko`, aby uniknąć redundancji danych.

### Zamówienia

Zamówienia stanowią niewielką (lecz kluczową) część modelu. Zgodnie z założeniami, zamówienie może być związane z wieloma (maksymalnie pięcioma) egzemplarzami. Egzemplarz może być przypisany do wielu zamówień, co tworzy relację wiele-do-wielu. Rozwiązałem to za pomocą tabeli pośredniczącej `ZamowienieEgzemplarz`. Ograniczenie do pięciu egzemplarzy jest niemożliwe w modelu ERD - wprowadziłem je później za pomocą wyzwalacza.

### Serwis

Reklamacje w moim modelu muszą być przypisane do jednego, konkretnego zamówienia, ale mogą obejmować wiele egzemplarzy. Nie wprowadziłem logiki wymuszającej, aby egzemplarz w reklamacji musiał być częścią przypisanego zamówienia. Można to osiągnąć poprzez odpowiedni wyzwalacz lub procedurę składowaną. Gdybym miał więcej czasu, zaimplementowałbym takie rozwiązanie. W celu uniknięcia redundancji wprowadziłem osobną encję `StatusReklamacji`.

### Sprzęt

Jeśli chodzi o sprzęt, zdecydowałem się na jedną tabelę dla całego sprzętu z atrybutem określającym, czy sprzęt jest profesjonalny. Uważałem, że **sprzętu profesjonalnego będzie znacząco mniej niż amatorskiego**, dlatego takie rozwiązanie wydaje się optymalne. Dodałem również pola `Rabat` i `Opis`, nawet dla sprzętu profesjonalnego, mimo że nie było to wymagane w projekcie.

W projekcie typ sprzętu nazwano "typem", ja jednak uznałem, że lepszym określeniem będzie "kategoria". Na przykład "górski" to zarówno typ, jak i kategoria sprzętu. W ten sposób można przypisać uprawnienia zarówno do sprzętu profesjonalnego, jak i do kategorii. Przyjąłem, że uprawnienia mogą nie wystąpić i może istnieć sprzęt lub kategoria bez przypisanych uprawnień.

### Kategoria

Kategorie w moim modelu mogą być przypisane do wielu pór roku, a pora roku do wielu kategorii. Relację wiele-do-wielu wyeliminowałem za pomocą tabeli pośredniczącej `KategoriaPoraRoku`. 

`Ranking` przypisałem w tabeli pośredniczącej pomiędzy `Sprzęt` a `Kategoria`, klasyfikując go jako należący do schematu `Sprzęt`. Uznałem to za bardziej sensowne z biznesowego punktu widzenia, ponieważ ranking przypisuje się do każdego sprzętu i kategorii.